<h1>Preparing the path for dataset</h1>

In [ ]:
!mkdir dataset
!mkdir dataset/yorkshire
!mkdir dataset/beagle
!mkdir dataset/pug
!mkdir dataset/retriever
!mkdir dataset/bulldog
!mkdir dataset/rottweiler
!mkdir dataset/pinscher
!mkdir dataset/tzu

<h1>Creating Functions and dealing with the images</h1>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import cv2
import matplotlib.pyplot as plt
import os

net = cv2.dnn.readNetFromCaffe("/kaggle/input/neuralnet/MobileNetSSD_deploy.prototxt.txt", "/kaggle/input/neuralnet/MobileNetSSD_deploy.caffemodel")
categories = ['yorkshire','beagle','pug','retriever','bulldog','rottweiler','pinscher','tzu']
imPath = 'dataset/yorkshire/n02094433_4028.jpg'
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]

def cropImage(imPath, destination):
    im = cv2.imread(imPath)
    blob = cv2.dnn.blobFromImage(cv2.resize(im, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()
    (h, w) = im.shape[:2]
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        idx = int(detections[0, 0, i, 1])
        if CLASSES[idx] == 'dog' or CLASSES[idx] == 'cat':
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                crop_img = im[startY:endY, startX:endX]
                try:
                    resized = cv2.resize(crop_img, (220,220), interpolation = cv2.INTER_AREA)
                    if resized.shape != None:
                        print("Salvo",destination)
                        cv2.imwrite(destination, resized)
                except Exception as e:
                    print("Erro", e)
        
def checkIfInCategories(categories, fileName):
    for c in categories:
        if c in fileName.lower():
            return True, c
    return False, None

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if '.jpg' in filename:
            file = os.path.join(dirname,filename)
            checked, fileDestination = checkIfInCategories(categories,file)
            if checked:
                destination = 'dataset/'+fileDestination+'/'+filename
                cropImage(file,destination)

<h1>Transfer Learning with ResNet</h1>

In [ ]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

num_classes = 8
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

<h1>Preparing train dataset</h1>

In [ ]:
image_size = 220
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory(
        'dataset',
        target_size=(image_size, image_size),
        class_mode='categorical')
answers = tuple(zip(train_generator.class_indices.values(), train_generator.class_indices.keys()))
my_new_model.fit_generator(train_generator,steps_per_epoch=100,epochs=7)

<h1>Saving the model </h1>

In [37]:
my_new_model.save('model.h5')

((0, 'beagle'),
 (1, 'bulldog'),
 (2, 'pinscher'),
 (3, 'pug'),
 (4, 'retriever'),
 (5, 'rottweiler'),
 (6, 'tzu'),
 (7, 'yorkshire'))

In [16]:
from IPython.display import FileLink
FileLink(r'modelo_final.h5')

/kaggle/working/kaggle/modelo_final.h5